In [ ]:
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
#return number of pages on olx site
def num_pages_on_site(url_string):
  base_html = requests.get(url_string).text
  soup = BeautifulSoup(base_html,'lxml',)
  strony = soup.find('div', class_="pager rel clr")
  if strony == None:
    return 1
  elif strony.find("fieldset",class_="fleft") == None:
    return  len(strony.find_all('span',class_='item fleft'))
  else:
    wiele_stron = strony.find("fieldset",class_="fleft")
    return int(re.search("totalPages: (\d+)",str(wiele_stron)).group(1))


In [ ]:
#input url string of olx page, if auctions on page return list of links
def auction_links_retrieval(url_string):
  base_html = requests.get(url_string).text
  soup = BeautifulSoup(base_html,'lxml')
  aukcje = soup.find_all('h3',class_="lheight22 margintop5")
  if aukcje == []: 
    return None
  else:
    links=[]
    for i in range(len(aukcje)):
      links.append(re.search('href="([a-zA-Z0-9:/.-]+)',str(aukcje[i])).group(1))
    return links

In [ ]:
# input url list, return dict of features
def feature_retrieval(url_list,feature_dict={},verbose=True):
  main_index = len(feature_dict)
  for url in url_list:
    if verbose:
      print(main_index,'parsing',url)
    if url.startswith('https://www.otomoto.pl'):
      otomoto_link = requests.get(url).text
      soup = BeautifulSoup(otomoto_link,'lxml')
      basic_otomoto = soup.find_all('li',class_='offer-params__item')
      main_index += 1
      if soup.find('span',class_="offer-price__number")!= None:
        cena = soup.find('span',class_="offer-price__number").text.strip().replace(" ","")
        feature_dict[main_index]={"Cena":cena}
      for i in range(len(basic_otomoto)):
        if basic_otomoto[i].a == None:
          value = basic_otomoto[i].div.text.strip()
        else:
          value = basic_otomoto[i].a.text.strip()
        label = basic_otomoto[i].span.text.strip()
        if main_index in feature_dict:
          feature_dict[main_index][label]=value
        else:          
          feature_dict[main_index] = {label:value}
    elif url.startswith('https://www.olx.pl'):
      olx_link = requests.get(url).text
      soup = BeautifulSoup(olx_link,'lxml')
      basic = soup.find_all('li',class_='css-ox1ptj')
      main_index += 1
      if soup.find('h3',class_='css-okktvh-Text eu5v0x0')!=None:
        cena = soup.find('h3',class_='css-okktvh-Text eu5v0x0').text
        feature_dict[main_index]={"Cena":cena}
      for index in range(len(basic)):
        if basic[index].text.endswith('Firmowe'):
          label = "Oferta od"
          value = 'Firmowe'
        elif basic[index].text.endswith('Prywatne'):
          label = "Oferta od"
          value = "Prywatne"
        else:
          label = re.search("(^\w+\.*\s*\w*)[:]\s((\d*\s*\w*)*|(\w+\s*\w*))",basic[index].text).group(1).strip()
          value = re.search("(^\w+\.*\s*\w*)[:]\s((\d*\s*\w*)*|(\w+\s*\w*))",basic[index].text).group(2).strip()
        if main_index in feature_dict:
          feature_dict[main_index][label] = value
        else:
          feature_dict[main_index] = {label:value}

  return feature_dict

In [ ]:
#Main program@@@@@@@@@@@@
#inputs marki = list of car brands, nadwozia = list of car bodies/
# od_roku = start from this production year, do_roku = end on this production year
def olx_car_scrapper(marki,nadwozia,od_roku,do_roku,feature_dict={},verbose=1):
  feature_dict=feature_dict
  count=0
  site_printer = False
  if verbose == 2:
    site_printer = True
  if verbose not in(1,2,3):
    return('Verbose has to be of type integer. And value 1, 2 or 3')
  if type(marki)!=list:
    return('Marki has to be of type list')
  if type(nadwozia)!=list:
    return('Nadwozia has to be of type list')  
  for index, marka in enumerate(marki):
    base_html = f"https://www.olx.pl/motoryzacja/samochody/{marki[index]}/"
    for rok in range(od_roku,do_roku+1):
      base_html+= f"?search%5Bfilter_float_year%3Afrom%5D={rok}&search%5Bfilter_float_year%3Ato%5D={rok}"
      for nadwozie in nadwozia:
        base_html+=f"&search%5Bfilter_enum_car_body%5D%5B0%5D={nadwozie}"
        num_pages = num_pages_on_site(base_html)
        auction_links = auction_links_retrieval(base_html)
        if num_pages == 1:
          print(f"number of pages on this site: {num_pages}")
          if auction_links == None:
            count+=1
            if verbose:
              print(f"Number of parsed pages:{count} Marka:{marka} Rok:{rok}")
          else:
            feature_dict = feature_retrieval(auction_links,feature_dict,verbose=site_printer)
            count+=1
            if verbose:
              print(f"Number of parsed pages:{count} Marka:{marka} Rok:{rok}")
        else:
          print(f"number of pages on this site: {num_pages}")
          feature_dict = feature_retrieval(auction_links,feature_dict,verbose=site_printer)
          count+=1
          if verbose:
              print(f"Number of parsed pages:{count} Marka:{marka} Rok:{rok}")
          for i in range(2,num_pages+1):
            base_html+=f"&page={i}"
            auction_links = auction_links_retrieval(base_html)
            feature_dict = feature_retrieval(auction_links,feature_dict,verbose=site_printer)
            count+=1
            if verbose:
              print(f"Number of parsed pages:{count} Marka:{marka} Rok:{rok}")            
  return feature_dict   

In [ ]:
##########TEST CASE##########
test_marka = ['toyota']
test_nadwozie = ['hatchback','sedan','estate-car']
test_od_roku = 2001
test_do_roku = 2001
test_features = olx_car_scrapper(test_marka,test_nadwozie,test_od_roku,test_do_roku,verbose=2)
print(test_features)

In [ ]:
#Create Pandas DataFrame from dictionary
import pandas as pd
test_df = pd.DataFrame.from_dict(test_features,orient="index")
test_df.head() #checking first 5 rows

In [ ]:
# The whole olx car dataset from 1995 to 2021
marki = ["aixam","alfa-romeo","audi","bmw","cadillac","chevrolet","chrysler","citroen","dacia","daewoo","daihatsu","dodge","fiat","ford","honda","hyundai","infiniti","jaguar","jeep",\
        "kia","lancia","landrover","lexus","mazda","mercedes-benz","mini","mitsubishi","nissan","opel","peugeot","polonez","porsche","renault","rover","saab","seat","skoda",\
        "smart","ssangyong","subaru","suzuki","toyota","volkswagen","volvo","pozostale"]
nadwozia = ["cabriolet",'sedan','coupe','pickup','hatchback','estate-car','off-road-vehicle','minibus','mvp','suv']
od_roku = 1995
do_roku = 2021
olx_dataset = olx_car_scrapper(marki,nadwozia,od_roku,do_roku,verbose=1)

In [ ]:
olx_df = pd.DataFrame.from_dict(olx_dataset,orient="index")
olx_df.head() #checking first 5 rows